In [1]:
import pandas as pd
import scipy as scp
import copy
from IPython.core.display import display, HTML
from Core.MeasuresManager import MeasuresManager

from IPython.display import clear_output
from ipywidgets import interact, interactive

import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display
from UI.ConverterUI import ConverterUI

# from scipy.stats import spearmanr
DATA_DIR = "data/"

In [2]:
data = pd.read_csv(DATA_DIR + 'seps_utf.csv', ';')
data = data[data.columns.drop(list(data.filter(regex='Unnamed')))]
data = data.drop(['Оценка', 'Группа'], 1)
data = data.dropna()
measures_manager = MeasuresManager(data, 'html')

measures_manager.raw_df

,Содержание дисциплины соответствовало заявленному в программе дисциплины,Место дисциплины в программе обучения обосновано,Учебная нагрузка дисциплины посильная,Уровень нагрузки в ходе обучения равномерный,Занятия проводились согласно расписанию,Доступ к материалам дисциплины предоставлялся вовремя,О заменах и переносах (при наличии таковых) сообщалось заранее в доступной форме,Перед изучением дисциплины [были ли у Вас соответствующие теоретические знания],Перед изучением дисциплины [был ли у Вас соответствующий практический опыт],"Оцените, пожалуйста, [приблизительный уровень Вашего присутствия на занятиях (в %)]",...,При изложении материала прослеживались связи [между этой и другими дисциплинами программы обучения],Преподаватель динамично реагировал на вопросы и потребности студентов,Готовы ли Вы охарактеризовать поведение уравновешенным и отношение к студентам объективным [говоря о преподавателе],Методы оценивания были понятны и обоснованы?,Вы получали вовремя результаты оценивания?,Давал ли преподаватель полезную обратную связь по результатам оценивания?,Методические указания содержали полезные сведения,Требуемый объем работ,Преподаватель эффективно помогал в решении возникавших проблем,Практические/лабораторные задания Вы выполняли
0,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Нет,Нет,0-30,...,"Да, регулярно",Да,Да,Да,Да,Да,"Однозначно, да",посильный,Всегда,с небольшой помощью
1,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Нет,Нет,90-100,...,Весьма эпизодически,Да,Да,Да,Да,Да,"Однозначно, да",посильный,Всегда,самостоятельно
2,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Не определился,Полностью согласен,Полностью согласен,Да,Нет,60-74,...,Весьма эпизодически,Да,Нет,Да,Да,Да,Некоторые рекомендации были полезны,посильный,Иногда,самостоятельно
3,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Да,Да,31-59,...,"Да, регулярно",Да,Да,Да,Да,Я не спрашивал,"Однозначно, да",посильный,Всегда,с небольшой помощью
4,Полностью согласен,Согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Да,Да,75-89,...,"Да, регулярно",Да,Да,Да,Да,Иногда баллы за электронные лабораторные работ...,"Однозначно, да",посильный,Всегда,с небольшой помощью
5,Согласен,Не определился,Согласен,Согласен,Согласен,Согласен,Согласен,Нет,Нет,75-89,...,Весьма эпизодически,Да,Да,Да,Да,Да,Некоторые рекомендации были полезны,посильный,Всегда,с небольшой помощью
6,Полностью согласен,Полностью согласен,Полностью согласен,Не определился,Полностью согласен,Полностью согласен,Согласен,Нет,Нет,0-30,...,"Да, регулярно",Да,Да,Да,Да,Да,"Однозначно, да",посильный,Всегда,с небольшой помощью
7,Полностью согласен,Не определился,Согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Да,Нет,90-100,...,Весьма эпизодически,Да,Да,Да,Да,Да,"Однозначно, да",посильный,Всегда,как команда
8,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Да,Нет,31-59,...,Весьма эпизодически,Да,Да,Да,Да,Да,"Однозначно, да",посильный,Всегда,самостоятельно
9,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Полностью согласен,Да,Нет,31-59,...,Весьма эпизодически,Да,Да,Да,Да,Да,Некоторые рекомендации были полезны,посильный,Всегда,самостоятельно


In [3]:
display(HTML(measures_manager.signs_html()))

Содержание дисциплины соответствовало заявленному в программе дисциплины,Полностью согласен – 38 Согласен – 12 Не определился – 1
Место дисциплины в программе обучения обосновано,Полностью согласен – 31 Согласен – 13 Не определился – 7
Учебная нагрузка дисциплины посильная,Полностью согласен – 38 Согласен – 11 Не определился – 1 Не согласен – 1
Уровень нагрузки в ходе обучения равномерный,Полностью согласен – 41 Согласен – 8 Не определился – 2
Занятия проводились согласно расписанию,Полностью согласен – 42 Согласен – 6 Не определился – 3
Доступ к материалам дисциплины предоставлялся вовремя,Полностью согласен – 43 Согласен – 8
О заменах и переносах (при наличии таковых) сообщалось заранее в доступной форме,Полностью согласен – 44 Согласен – 5 Не определился – 2
Перед изучением дисциплины [были ли у Вас соответствующие теоретические знания],Нет – 29 Да – 22
Перед изучением дисциплины [был ли у Вас соответствующий практический опыт],Нет – 37 Да – 14
"Оцените, пожалуйста, [приблизительный уровень Вашего присутствия на занятиях (в %)]",0-30 – 16 31-59 – 12 75-89 – 10 90-100 – 8 60-74 – 5
"Оцените, пожалуйста, [свою успешность в дисциплине (в баллах)]",90-100 – 28 75-89 – 16 60-74 – 5 31-59 – 2


In [9]:
# Experiments with widgets. Ignore this cell
# from ipywidgets import widgets


# text_wid = widgets.Text()
# # display(text_wid)

# def text_handler(sender):
#     print(text_wid.value)

# text_wid.on_submit(text_handler)

In [4]:
display(HTML(measures_manager.signs))
signs_dropdown = widgets.Dropdown(
    options=measures_manager.raw_signs_names(),
    value=measures_manager.raw_signs_names()[0],
    description='Choose sign: ',
    disabled=False,
    style={'description_width': 'initial', 'width': '500px'},
    layout={'width':'100%'}
)
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output(wait=True)
        display(signs_dropdown)
        display(HTML(measures_manager[change['new']].get_stat_info()))

signs_dropdown.observe(on_change)
display(signs_dropdown)
display(HTML(measures_manager.signs[0].get_stat_info()))


Dropdown(description='Choose sign: ', index=2, layout=Layout(width='100%'), options=('Содержание дисциплины со…

TypeError: 'NoneType' object is not iterable

In [11]:
change_entity = dict()
convert_container = widgets.VBox()
execute_button = widgets.Button(description='Click me',
    disabled=True
    )
disabled_func = lambda:(not((len(convert_container.children) != 0) and (lambda arr: len(set(arr)) == len(arr))(list(map(lambda drp: drp.value, convert_container.children)))))
def update_btn():
    print('len? ', len(convert_container.children) != 0)
    print('uniq? ', (lambda arr: len(set(arr)) == len(arr))(list(map(lambda drp: drp.value, convert_container.children))))
    print('enable? ', disabled_func())
    execute_button.disabled = disabled_func()
    
signs_dropdown = widgets.Dropdown(
    options=measures_manager.raw_signs_names(),
    value=None,
    description='Choose sign: ',
    disabled=False,
    style={'description_width': 'initial', 'width': '500px'},
    layout={'width':'100%'}
)
def on_measure_change(change):
    res_array = []
    if change['type'] == 'change' and change['name'] == 'value':
        print('len? ', len(convert_container.children) != 0)
        print('uniq? ', (lambda arr: len(set(arr)) == len(arr))(list(map(lambda drp: drp.value, convert_container.children))))
        print('is enable: ', (len(convert_container.children) != 0) and (lambda arr: len(set(arr)) == len(arr))(list(map(lambda drp: drp.value, convert_container.children))))
        execute_button.disabled = disabled_func()
        print('btn ststus', execute_button.disabled)
#         convert_container.children = ()
#         convert_container.children += (widgets.Label(value=signs_dropdown.value),)
        i = 0
        change_entity['measure'] = change['new']
        if change['new'] == 'Ordinal':            
            print(measures_manager[signs_dropdown.value]._aggregated_data)
            sign_values = measures_manager[signs_dropdown.value]._aggregated_data.keys()
            for val in sign_values:
                i += 1
                drpdpwn = widgets.Dropdown(value=None, description=f'{i}. ', options=sign_values)
                drpdpwn.observe(lambda x:(update_btn()))
                convert_container.children += (drpdpwn,)
                

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        chosen_sign = measures_manager[change['new']]
        change_entity['sign'] = change['new']
        measures = ['Nominal', 'Ordinal', 'Interval']
        clear_output(wait=True)
        display(signs_dropdown)
        measures.remove(chosen_sign.measure)
        display(HTML(f"<div>Current measure: <span style='font-weight: bold;'>{chosen_sign.measure}</span></div>"))
        allowed_measures_dropdown = widgets.Dropdown(
            options=measures,
            value=None,
            description=f'Change measure from <b><u>{chosen_sign.measure}</u></b> to:',
            disabled=False,
            style={'description_width': 'initial', 'font_size':'20px'},
            layout={'width':'100%', 'height': '37px'}
        )
        allowed_measures_dropdown.observe(on_measure_change)
        display(allowed_measures_dropdown)
        display(convert_container)
        display(execute_button)
#         display(HTML(measures_manager[change['new']].get_stat_info()))

signs_dropdown.observe(on_change)

display(signs_dropdown)
print(signs_dropdown.value)

# display(HTML(measures_manager.signs[0].get_stat_info()))

Dropdown(description='Choose sign: ', layout=Layout(width='100%'), options=('Содержание дисциплины соответство…

None


In [3]:

converter_ui = ConverterUI(measures_manager)
converter_ui.ui()

Dropdown(description='Choose sign: ', index=2, layout=Layout(width='100%'), options=('Содержание дисциплины со…

Dropdown(description='Change measure from <b><u>Nominal</u></b> to:', layout=Layout(height='37px', width='100%…

VBox()

Button(description='Convert measure', disabled=True, style=ButtonStyle())

op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка дисциплины посильная', 'measure': 'Ordinal'}
op data:  {'sign_name': 'Учебная нагрузка д